## get parcel data per apn
provides useful info, especially for development.

does not have any ownership info

In [132]:
import pandas as pd
import time,pathlib,re
import scipy as sp
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [154]:
# get a list of apns
if(1):
    sheetname = "data"
    df = pd.read_excel(r"\\NAS-SALEM\Documents\real estate\sd-tax-auction\2021-sd-tax-auction\2-9-2021 Auction List.xls",sheet_name=sheetname)
    apns = df["APN"]
    filterKey   = "TimeShare"
    filterValue = False
else:
    apns= ["123563454"]

# grab a sub set    
apns = apns[0:152]
#apns=[2931214100]
N= sp.size(apns)


In [152]:
resultFolder = pathlib.Path(r"\\NAS-SALEM\work\data\sd-auction-2021")
resultFolder.mkdir(parents=True, exist_ok=True)


tmpFileName= "{APN}.sangis.html"


In [153]:
idx = 0
for apn in apns:
    print("%d (%2.2f) -->%d"%(idx,100.0 *(idx+1)/N,apn));
    idx+=1
    # download the executable
    driver  = webdriver.Opera(executable_path=r"C:\src\operadriver_win64\operadriver.exe")
    driver.implicitly_wait(30) # seconds
    url =  'https://gissd.sandag.org/plt/ParcelRpt.aspx?APN={APN}'.format(APN=apn)
    driver.get(url)
    time.sleep(10)
    data = driver.page_source
    driver.close()

    with resultFolder.joinpath(tmpFileName.format(APN=apn)).open('w') as f:
        f.write(data);
print("done")   

0 (100.00) -->2931214100
done


# parse html pages saved

now get all the data from html pages and create a composite csv file with one row per apn

In [139]:
mapper = {
#'Property Characteristics:',
'Baths:'                      :3,
'Assessor Land:'              :2,
'Improvements:'               :7,
#'Add. Areas:',
#'Tot. Living Area:',
'Assessor Total:'              :9 ,
'Tax Status:'                 :14,  
'Acreage:'                    :15,
'Bedrooms:'                   :18,
'Tax Rate Area:'              :17,      
 #'District Information:',
 #'City Council:',
 #'Board of Supervisors:',
 #'School District:',
 #'School Union:',
 #'Congress:',
 #'Fire Protection:',
 #'Senate:',
 #'Assembly:',
 #'Community College:',
 #'Planning Areas:',
 #'Census Tract:',
'Subregional Area:'           :67,
 #'Major Statistical Area:',
 #'Community Planning Area:',
'Land Use:'                   :slice(74,75),
'Existing (LU):'              :slice(78,79),
 #'Ecology:',
 #'Vegetation Type (County):',
 #'Vegetation Type (Western Region 2012):',
'Flood Zone/Flood Plain:'     :[89,91]
 #'Geology Type:',
 #'Soil Type:'
}

def processHtml(apn):
    with resultFolder.joinpath(tmpFileName.format(APN=apn)).open('r') as f:
        #try:    
            #f=open(file,'r')
        data = f.read()
            #f.close()
    #except:
    #    print("cant find html for :%s"%(file))
    #    return -1
    soup = BeautifulSoup(data)
  # save a pretty version
    if(0):
        f = open("test.html",'w')
        f.write(soup.prettify());
        f.close()
                               
    ELEM_SEPERATOR = '|'
    ID_IDENTIFIER  = ':'
    s = soup.get_text(ELEM_SEPERATOR)
    startI   = int(s.find("Property Characteristics"))
    endI     = int(s.find("Data Disclaimer"))
    content  = s[startI:endI]
    contentS =content.split(ELEM_SEPERATOR)
    contentId=[c.strip() for c in contentS if ID_IDENTIFIER in c]
    return contentS;                               

In [155]:
f = resultFolder.joinpath("data.csv").open('w')
str = "APN,type,";
for key, value in mapper.items():
  str +=key+','
str+="LU code,LU Description,PLU Code,PLU Descriptio,Land type info"
f.write(str+"\n");

sheets =["data"]
filterKey   = "TimeShare"
filterValue = False

sheets =["data"]
for apn in apns:
    content = processHtml(apn)
    if(content==-1):continue
    str = "%s,%s,"%(apn,sheetname);
    for key, value in mapper.items():
        if(isinstance(value,slice)):
            entry =" ".join(content[value])
        elif(isinstance(value,list)):
            entry =" ".join([content[v] for v in value])
        else:
            entry =content[value]
        str += entry.replace(',','')+','
    flat = "".join(content);
    found = re.search("Land Use.*?LU\)\:([0-9]*)(.*?)\(PLU\)([0-9]*)(.*?)Ecology",flat)
    str +=",".join(found.groups())+","+found.group (0);
    f.writelines(str+"\n")
    
f.close()
